# 02) 뉴스 감성 정렬 및 병합 — TSLA vs F (2025)

## ToC
1. GDELT 기사 로드 및 텍스트 정제
2. VADER 감성 점수 계산
3. 일자별 집계 (평균/가중평균)
4. yfinance 주가 수익률 계산 (동일일, T+1, T+2)
5. 병합 및 상관/회귀/그랜저 인과성(선택)
6. 시각화 및 해석

In [ ]:

import pandas as pd, numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from dateutil import parser

analyzer = SentimentIntensityAnalyzer()

# TODO: 파일 경로 수정
news_paths = {
    "TSLA": "../data/raw/news_tsla_2025.json",
    "F": "../data/raw/news_ford_2025.json",
}
prices_path = "../data/raw/prices.csv"

def load_gdelt_articles(path):
    import json
    with open(path, "r", encoding="utf-8") as f:
        d = json.load(f)
    arts = d.get("articles") or d.get("documents") or []
    rows = []
    for a in arts:
        title = a.get("title") or ""
        desc  = a.get("seendate") or ""
        url   = a.get("url") or a.get("sourceCommonName") or ""
        dtstr = a.get("seendate") or a.get("date")
        try:
            dt = parser.parse(dtstr).date()
        except Exception:
            continue
        score = analyzer.polarity_scores(title).get("compound", 0.0)
        rows.append({"date": dt, "title": title, "url": url, "sentiment": score})
    return pd.DataFrame(rows)

dfs = []
for tkr, p in news_paths.items():
    df = load_gdelt_articles(p)
    df["ticker"] = tkr
    dfs.append(df)
news = pd.concat(dfs, ignore_index=True)
news_daily = news.groupby(["ticker", "date"])["sentiment"].mean().reset_index()

prices = pd.read_csv(prices_path, parse_dates=["Date"])
prices = prices.rename(columns={"Date":"date"}).set_index("date")
ret = prices.pct_change().dropna()
ret = ret.stack().rename("ret").reset_index().rename(columns={"level_1":"ticker"})

# 정렬: 뉴스 감성 (t) -> 주가 수익률 (t+1)
news_daily["date_t1"] = pd.to_datetime(news_daily["date"]) + pd.Timedelta(days=1)
merge = pd.merge(
    news_daily.rename(columns={"date_t1":"date"})[["ticker","date","sentiment"]],
    ret, on=["ticker","date"], how="inner"
)
print(merge.head())

print("상관계수 by ticker:")
print(merge.groupby("ticker")[["sentiment","ret"]].corr().unstack().iloc[:,1])
